In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [23]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func , inspect
import pandas as pd

In [2]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [3]:
# inspect database 

inspector = inspect(engine)
inspector.get_table_names()


['measurement', 'station']

In [4]:

# reflect an existing database into a new model(use automap)
Base = automap_base()


# reflect the tables
Base.prepare(engine, reflect = True)


In [5]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [6]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [7]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [19]:
# find all the columns and datatypes in the measurement table 
inspector = inspect(engine)
inspector.get_table_names()
# Get a list of column names and types
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [21]:
# station table
# Get a list of column names and types
columns = inspector.get_columns('Station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [35]:
# Find the most recent date in the data set.
max_date = engine.execute('SELECT max(date) FROM measurement').fetchall()
max_date

mydate = func.to_date(engine.execute('SELECT max(date) FROM measurement').fetchall(),'YYYY-MM-DD')
mydate

<sqlalchemy.sql.functions.Function at 0x7f891bfabf28; to_date>

In [28]:
import datetime
import dateutil.relativedelta
d = datetime.datetime.strptime(max_date,"%Y-%m-%d")

TypeError: strptime() argument 1 must be str, not list

In [26]:
# use pandas to figure out max date - 12 months
pandas_max_date = pd.Timestamp(max_date)
pandas_max_date

#date_list = pd.date_range('1969-12-01','2021-08-01', 
 #             freq='W').strftime("%Y-%m-%d").tolist()



TypeError: Cannot convert input [[('2017-08-23',)]] of type <class 'list'> to Timestamp

In [28]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [7]:
# Close Session
session.close()

NameError: name 'session' is not defined